In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mlp_keras import MLP
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
folder = '../report/NO_OS/postop_binary'
file = '/data.csv'

data = pd.read_csv(folder + file)

X, y = data.iloc[:,:-1], data.iloc[:, -1]
feature_names, target_names = data.columns[:-1].to_list(), data.columns[-1:].to_list()

print(X[:5])
print(y[:5])
print(feature_names)
print(target_names)

   Age at Operation  Gender  Days Abdominal Pain  Temperature  Tenderness  \
0                15       1                    1         36.9           0   
1                14       0                    2         37.0           0   
2                16       0                    0         36.9           1   
3                14       0                    1         35.7           1   
4                16       1                    7         37.9           0   

   Leukos [10^9/l]  CRP [mg/l]  No of Sonographies  Diameter Appendix [mm]  
0            11.10         6.0                 1.0                     8.0  
1            21.28        30.8                 1.0                    10.0  
2            13.07         1.9                 1.0                     8.0  
3             7.95         0.0                 2.0                     4.0  
4            14.82        14.4                 3.0                     8.0  
0    0
1    1
2    0
3    0
4    0
Name: Postoperative Diagnosis, dtype: in

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
lr = RandomForestClassifier(n_estimators=100, random_state=42)
lr.fit(X_train, y_train)
accuracy_score(y_test, lr.predict(X_test))

0.9064748201438849

In [5]:
mlp = MLP(folder + file)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, restore_best_weights=True)
rlr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=0.0000001)

mlp.train(
    9,
    2,
    'Adam',
    0.01,
    [early_stopping_callback, rlr_callback],
    dropout=0.24,
    save_folder=None,
    oversampling=False,
    test=False
)

print(mlp.trained_model)

Epoch 1/300
28/28 [==============================] - 4s 30ms/step - loss: 4.2607 - accuracy: 0.7410 - val_loss: 0.4445 - val_accuracy: 0.8304 - lr: 0.0100
Epoch 2/300
28/28 [==============================] - 0s 10ms/step - loss: 0.4369 - accuracy: 0.8198 - val_loss: 0.4419 - val_accuracy: 0.8125 - lr: 0.0100
Epoch 3/300
28/28 [==============================] - 0s 9ms/step - loss: 0.3779 - accuracy: 0.8311 - val_loss: 0.5114 - val_accuracy: 0.8393 - lr: 0.0100
Epoch 4/300
28/28 [==============================] - 0s 12ms/step - loss: 0.4172 - accuracy: 0.8581 - val_loss: 0.4118 - val_accuracy: 0.8304 - lr: 0.0100
Epoch 5/300
28/28 [==============================] - 0s 10ms/step - loss: 0.4041 - accuracy: 0.8311 - val_loss: 0.4050 - val_accuracy: 0.8393 - lr: 0.0100
Epoch 6/300
28/28 [==============================] - 0s 9ms/step - loss: 0.3434 - accuracy: 0.8671 - val_loss: 0.3999 - val_accuracy: 0.8304 - lr: 0.0100
Epoch 7/300
28/28 [==============================] - 0s 9ms/step - loss:

In [ ]:
from PyALE import ale

df = data.iloc[:, :-1]
print(lr.predict(df)[:10])
print(mlp.predict(df)[:10])

for i, f in enumerate(df.columns.to_list()):
    ale_eff = ale(
        X=df,
        model=lr,
        feature=[f],
        grid_size=10,
        include_CI=False
    )

In [10]:
mlp.trained_model.save("my_tf_model")
loaded_model = tf.keras.models.load_model("my_tf_model")
loaded_model.predict(X_test.iloc[[0]])
# import shap

# shap_explainer = shap.DeepExplainer(loaded_model, X_train)
# shap_values = shap_explainer.shap_vaues(X_test.iloc[[0]], check_additivity=False)
# shap.summary_plot(shap_values, X_test.iloc[[0]])

INFO:tensorflow:Assets written to: my_tf_model\assets


tensorflow:INFO: Assets written to: my_tf_model\assets
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


AttributeError: Exception encountered when calling layer 'sequential' (type Sequential).

'tuple' object has no attribute 'rank'

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=['     Age at Operation  Gender  Days Abdominal Pain  Temperature  Tenderness  \\\n110                10       0                    1         37.6           0   \n82                 14       0                    1         36.8           1   \n51                 13       0                    1         36.6           0   \n218                 1       0                    1         38.9           2   \n557                 7       0                    2         37.2           0   \n..                ...     ...                  ...          ...         ...   \n71                 13       0                    1         36.7           0   \n106                12       0                    1         37.5           0   \n270                13       1                    1         36.4        